# WaPOR v3 API Tutorial

In this notebook we'll have a look at how to download WaPOR v3 data using FAO's [GISMGR 2.0 API](https://github.com/un-fao/FAO-Water-Applications/blob/10a70cda0e0023258bc9152e43a73b6e8e20b669/WaPOR/FAO%20GISMGR%202.0%20-%20API%20Reference.pdf) for a specific region (for a notebook on downloading WaPOR v2 data, go [here](https://colab.research.google.com/github/un-fao/FAO-Water-Applications/blob/main/WaPOR/WaPORv2_API.ipynb)). Let's get started by importing the modules we are going to use.

In [1]:
# import ee
import re
import requests
import numpy as np
import matplotlib.pyplot as plt
from osgeo import gdal

import rasterio
import geopandas as gpd
import datetime
import pandas as pd


 the base URL and a function that requests the url and returns some of the information thats returned.

In [2]:
base_url = f"https://data.apps.fao.org/gismgr/api/v2/catalog/workspaces/WAPOR-3/mapsets"
def collect_responses(url, info = ["code"]):
    data = {"links": [{"rel": "next", "href": url}]}
    output = list()
    while "next" in [x["rel"] for x in data["links"]]:
        url_ = [x["href"] for x in data["links"] if x["rel"] == "next"][0]
        response = requests.get(url_)
        response.raise_for_status()
        data = response.json()["response"]
        if isinstance(info, list):
            output += [tuple(x.get(y) for y in info) for x in data["items"]]
        else:
            output += data["items"]
    if isinstance(info, list):
        output = sorted(output)
    return output

chose a mapset code to get links to all raster of that mapset

In [10]:
# Read points from shapefile
fh = r"../0_insitu/FLUXNET2022.csv"
pts = pd.read_csv(fh,sep=';')
pts = pts[pts['x'].notna()]
coords = [(x,y) for x, y in zip(pts.x, pts.y)]

In [11]:
pts

,Unnamed: 0,Site,Period,Dataset,x,y
0,0,AR-TF1,2016-2018,AMF,-66.733500,-54.973300
1,1,CA-Cbo,1994-2020,AMF,-79.933300,44.316700
2,2,CA-DB2,2019-2020,AMF,-122.995100,49.119000
3,3,CA-DBB,2014-2020,AMF,-122.984900,49.129300
4,4,CA-ER1,2015-2020,AMF,-80.412300,43.640500
...,...,...,...,...,...,...
267,267,SE-Deg,2001-2020,ICOS-WW,19.557448,64.182333
268,268,SE-Htm,2015-2020,ICOS-WW,13.417893,56.097566
269,269,SE-Nor,2014-2020,ICOS-WW,17.478520,60.086990
270,270,SE-Ros,2014-2020,ICOS-WW,19.738000,64.172500


## Monthly Data

In [14]:
mapset_code = "L1-AETI-M"
mapset_url = f"{base_url}/{mapset_code}/rasters"
all_rasters = collect_responses(mapset_url, info = ["code", "downloadUrl"])
raster_urls = [i[1] for i in all_rasters]

In [20]:
raster_urls

['https://storage.googleapis.com/fao-gismgr-wapor-3-data/DATA/WAPOR-3/MAPSET/L1-AETI-M/WAPOR-3.L1-AETI-M.2018-01.tif',
 'https://storage.googleapis.com/fao-gismgr-wapor-3-data/DATA/WAPOR-3/MAPSET/L1-AETI-M/WAPOR-3.L1-AETI-M.2018-02.tif',
 'https://storage.googleapis.com/fao-gismgr-wapor-3-data/DATA/WAPOR-3/MAPSET/L1-AETI-M/WAPOR-3.L1-AETI-M.2018-03.tif',
 'https://storage.googleapis.com/fao-gismgr-wapor-3-data/DATA/WAPOR-3/MAPSET/L1-AETI-M/WAPOR-3.L1-AETI-M.2018-04.tif',
 'https://storage.googleapis.com/fao-gismgr-wapor-3-data/DATA/WAPOR-3/MAPSET/L1-AETI-M/WAPOR-3.L1-AETI-M.2018-05.tif',
 'https://storage.googleapis.com/fao-gismgr-wapor-3-data/DATA/WAPOR-3/MAPSET/L1-AETI-M/WAPOR-3.L1-AETI-M.2018-06.tif',
 'https://storage.googleapis.com/fao-gismgr-wapor-3-data/DATA/WAPOR-3/MAPSET/L1-AETI-M/WAPOR-3.L1-AETI-M.2018-07.tif',
 'https://storage.googleapis.com/fao-gismgr-wapor-3-data/DATA/WAPOR-3/MAPSET/L1-AETI-M/WAPOR-3.L1-AETI-M.2018-08.tif',
 'https://storage.googleapis.com/fao-gismgr-wapo

In [21]:
# Downlaod time series data 
scale = 0.1
urls = raster_urls # chose raster from which you want to downlaod the pixel data 
pt_data = []
dates = []
for url in urls:
    date = pd.to_datetime(url[-11:-4]) 
    dates.append(date)
    # Open the raster and store metadata
    src = rasterio.open(url)

    # Sample the raster at every point location and store values in DataFrame
    pt = [x[0] for x in src.sample(coords)]
    pt_data.append(pt)
    
# convert the data to adataframe
df = pd.DataFrame(pt_data, columns=pts.Site, index = dates)
df = df*scale


In [24]:
df.to_csv('Data/FLUXNET2022_L1_AETI_M.csv')

## Dekadal data

In [3]:
mapset_code = "L1-AETI-D"
mapset_url = f"{base_url}/{mapset_code}/rasters"
all_rasters = collect_responses(mapset_url, info = ["code", "downloadUrl"])
raster_urls = [i[1] for i in all_rasters]
raster_urls

['https://storage.googleapis.com/fao-gismgr-wapor-3-data/DATA/WAPOR-3/MAPSET/L1-AETI-D/WAPOR-3.L1-AETI-D.2018-01-D1.tif',
 'https://storage.googleapis.com/fao-gismgr-wapor-3-data/DATA/WAPOR-3/MAPSET/L1-AETI-D/WAPOR-3.L1-AETI-D.2018-01-D2.tif',
 'https://storage.googleapis.com/fao-gismgr-wapor-3-data/DATA/WAPOR-3/MAPSET/L1-AETI-D/WAPOR-3.L1-AETI-D.2018-01-D3.tif',
 'https://storage.googleapis.com/fao-gismgr-wapor-3-data/DATA/WAPOR-3/MAPSET/L1-AETI-D/WAPOR-3.L1-AETI-D.2018-02-D1.tif',
 'https://storage.googleapis.com/fao-gismgr-wapor-3-data/DATA/WAPOR-3/MAPSET/L1-AETI-D/WAPOR-3.L1-AETI-D.2018-02-D2.tif',
 'https://storage.googleapis.com/fao-gismgr-wapor-3-data/DATA/WAPOR-3/MAPSET/L1-AETI-D/WAPOR-3.L1-AETI-D.2018-02-D3.tif',
 'https://storage.googleapis.com/fao-gismgr-wapor-3-data/DATA/WAPOR-3/MAPSET/L1-AETI-D/WAPOR-3.L1-AETI-D.2018-03-D1.tif',
 'https://storage.googleapis.com/fao-gismgr-wapor-3-data/DATA/WAPOR-3/MAPSET/L1-AETI-D/WAPOR-3.L1-AETI-D.2018-03-D2.tif',
 'https://storage.google

In [12]:
# Downlaod time series data 
scale = 0.1
urls = raster_urls # chose raster from which you want to downlaod the pixel data 
pt_data = []
dates = []
dekad_day = {
   'D1':'01',
    'D2': '11',
    'D3': '21'    
}

for url in urls:
    date = pd.to_datetime(url[-14:-6]+dekad_day[url[-6:-4]]) 
    dates.append(date)
    # Open the raster and store metadata
    src = rasterio.open(url)

    # Sample the raster at every point location and store values in DataFrame
    pt = [x[0] for x in src.sample(coords)]
    pt_data.append(pt)
    
# convert the data to adataframe
df = pd.DataFrame(pt_data, columns=pts.Site, index = dates)
df = df*scale
df.to_csv('Data/FLUXNET2022_L1_AETI_D_2025-10-24].csv')


RasterioIOError: Read or write failed. WAPOR-3.L1-AETI-D.2024-12-D2.tif, band 1: IReadBlock failed at X offset 433, Y offset 168: _TIFFPartialReadStripArray:Cannot read offset/size for strile around ~81073

In [37]:
df

Site,AR-TF1,CA-Cbo,CA-DB2,CA-DBB,CA-ER1,CA-LP1,PE-QFR,US-ALQ,US-ARM,US-Bar,...,IT-Ren,IT-SR2,IT-Tor,RU-Fy2,RU-Fyo,SE-Deg,SE-Htm,SE-Nor,SE-Ros,SE-Svb
2018-01-01,1.6,0.0,0.1,0.1,0.0,0.0,1.6,0.0,0.0,0.0,...,0.0,0.6,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0
2018-01-01,1.7,0.0,0.2,0.1,0.0,0.0,1.6,0.0,0.0,0.0,...,0.0,0.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018-01-01,1.8,0.0,0.2,0.2,0.0,0.0,2.1,0.0,0.5,0.0,...,0.1,0.6,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0
2018-01-01,1.5,0.0,0.1,0.1,0.0,0.0,2.6,0.0,0.2,0.0,...,0.1,0.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018-01-01,1.5,0.0,0.1,0.1,0.0,0.0,2.4,0.0,0.3,0.1,...,0.1,0.8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-01-01,1.1,0.9,0.3,0.2,1.1,0.0,1.9,0.9,1.1,0.5,...,0.4,1.2,0.2,0.0,0.0,0.1,0.2,0.0,0.1,0.1
2018-01-01,1.1,0.5,0.1,0.1,0.5,0.0,2.7,0.3,0.2,0.3,...,0.6,1.3,0.2,0.0,0.0,0.0,0.0,0.0,0.1,0.1
2018-01-01,1.3,0.2,0.2,0.1,0.1,0.0,2.4,0.0,0.5,0.2,...,0.3,1.0,0.1,0.0,0.0,0.0,0.1,0.0,0.0,0.0
2018-01-01,1.7,0.0,0.1,0.0,0.0,0.0,2.9,0.0,0.3,0.1,...,0.1,0.9,0.0,0.0,0.0,0.0,0.2,0.0,0.1,0.3


# dekadal E, T, I

In [4]:
# Read points from shapefile
shapefile = r"../../GIS/FLUXNET2022.shp"
pts = gpd.read_file(shapefile)
pts = pts = pts[pts['x'].notna()]
coords = [(x,y) for x, y in zip(pts.x, pts.y)]

In [6]:
codes=[
    "L1-E-D",
    "L1-T-D",
    "L1-I-D"
]
# mapset_code = "L1-AETI-M"
for mapset_code in codes:
    mapset_url = f"{base_url}/{mapset_code}/rasters"
    all_rasters = collect_responses(mapset_url, info = ["code", "downloadUrl"])
    raster_urls = [i[1] for i in all_rasters]
    
    # Downlaod time series data 
    scale = 0.1
    urls = raster_urls # chose raster from which you want to downlaod the pixel data 
    pt_data = []
    dates = []
    dekad_day = {
       'D1':'01',
        'D2': '11',
        'D3': '21'    
    }
    
    for url in urls:
        date = pd.to_datetime(url[-14:-6]+dekad_day[url[-6:-4]]) 
        dates.append(date)
        # Open the raster and store metadata
        src = rasterio.open(url)
    
        # Sample the raster at every point location and store values in DataFrame
        pt = [x[0] for x in src.sample(coords)]
        pt_data.append(pt)
        
    # convert the data to adataframe
    df = pd.DataFrame(pt_data, columns=pts.Site, index = dates)
    df = df*scale
    df.to_csv(f'Data/FLUXNET2022_{mapset_code}.csv')

In [7]:
raster_urls

['https://storage.googleapis.com/fao-gismgr-wapor-3-data/DATA/WAPOR-3/MAPSET/L1-I-D/WAPOR-3.L1-I-D.2018-01-D1.tif',
 'https://storage.googleapis.com/fao-gismgr-wapor-3-data/DATA/WAPOR-3/MAPSET/L1-I-D/WAPOR-3.L1-I-D.2018-01-D2.tif',
 'https://storage.googleapis.com/fao-gismgr-wapor-3-data/DATA/WAPOR-3/MAPSET/L1-I-D/WAPOR-3.L1-I-D.2018-01-D3.tif',
 'https://storage.googleapis.com/fao-gismgr-wapor-3-data/DATA/WAPOR-3/MAPSET/L1-I-D/WAPOR-3.L1-I-D.2018-02-D1.tif',
 'https://storage.googleapis.com/fao-gismgr-wapor-3-data/DATA/WAPOR-3/MAPSET/L1-I-D/WAPOR-3.L1-I-D.2018-02-D2.tif',
 'https://storage.googleapis.com/fao-gismgr-wapor-3-data/DATA/WAPOR-3/MAPSET/L1-I-D/WAPOR-3.L1-I-D.2018-02-D3.tif',
 'https://storage.googleapis.com/fao-gismgr-wapor-3-data/DATA/WAPOR-3/MAPSET/L1-I-D/WAPOR-3.L1-I-D.2018-03-D1.tif',
 'https://storage.googleapis.com/fao-gismgr-wapor-3-data/DATA/WAPOR-3/MAPSET/L1-I-D/WAPOR-3.L1-I-D.2018-03-D2.tif',
 'https://storage.googleapis.com/fao-gismgr-wapor-3-data/DATA/WAPOR-3/MA